In [1]:
# importing the packages
import cv2
import numpy as np


# writting functions

def stats(image):
#     split the channels of image to l, a, b
    (l,a,b) = cv2.split(image)
#     calculate the mean and standard deviation of each channel
    (lMean, lStd) = (l.mean(), l.std())
    (aMean, aStd) = (a.mean(), a.std())
    (bMean, bStd) = (b.mean(), b.std())
#     return the values
    return (lMean, lStd, aMean, aStd, bMean, bStd)


def clipping(arr):
#     only keep those values which are in the range of rgb color space
    arr = np.clip(arr, 0 , 255)
    return arr



def change_color(source, target):
#     convert the bgr to lab color space
    source = cv2.cvtColor(source, cv2.COLOR_BGR2LAB).astype("float32")
    target = cv2.cvtColor(target, cv2.COLOR_BGR2LAB).astype("float32")
    
    (lMeansrc, lStdsrc, aMeansrc, aStdsrc, bMeansrc, bStdsrc) = stats(source)
    (lMeantar, lStdtar, aMeantar, aStdtar, bMeantar, bStdtar) = stats(target)

#     get the l,a, b from the target image
    (l, a, b) = cv2.split(target)
    
#     subtract the target mean from the (l, a, b) respectively
    l -= lMeantar
    a -= aMeantar
    b -= bMeantar
    
    
    l = (lStdtar/lStdsrc)*l
    a = (aStdtar/aStdsrc)*a
    b = (bStdtar/bStdsrc)*b
    
    l += lMeansrc
    a += aMeansrc
    b += bMeansrc
    
#     now clip the (l, a, b)
    l = clipping(l)
    a = clipping(a)
    b = clipping(b)
    
#     merge the (l, a, b) values 
    transfer = cv2.merge([l, a, b])
#     convert the colorspace from lab to bgr and type from float32 to int8 which is default
    transfer = cv2.cvtColor(transfer.astype("uint8"), cv2.COLOR_LAB2BGR)
#     return the modified image
    return transfer
    
    

    
# for showing the images on the screen we should resize them. The following function will resize the image
# taking the width as 400.
# if the current width of the image is lesser than 400 then it will increase the width of the original image
# and if the width of the original image is greater than 400 then it will decrease it to 400
    
def show_image(title, image, width = 400):
#   set width as 500 and calculate the ratio of new and old width
    ratio = width / float(image.shape[1])
    
#     we have to change the height according to ratio
    dim = (width, int(image.shape[0] * ratio))
    
#     now new (width, height) should be dim
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    # show the resized image
    cv2.imshow(title, resized)
    
    
    
# load the images in source and target
# source-> the image from which you want to pick color and feel
# target -> image which you want to modify according to the color and feel of the source image

source = cv2.imread("H:/Workspace/images/sunset.jpg")    
target = cv2.imread("H:/Workspace/images/Sunrise.jpg")

# the resultant image
transfer = change_color(source, target)


# calling function to show the images
show_image("source", source)
show_image("target", target)
show_image("transfer", transfer)
# cv2.imshow("image", transfer.resize(transfer, 300 ) )
cv2.waitKey(0)
cv2.destroyAllWindows()